In [62]:

import requests
import random
import os
import pandas as pd
from rdkit import Chem

upload_url = r"http://xundrug.cn:5001/modules/upload0/"


def predict_pka(smi):
    param = {"Smiles": ("tmg", smi)}
    headers = {"token": "O05DriqqQLlry9kmpCwms2IJLC0MuLQ7"}
    response = requests.post(url=upload_url, files=param, headers=headers)
    # return response
    jsonbool = int(response.headers["ifjson"])
    if jsonbool == 1:
        res_json = response.json()
        if res_json["status"] == 200:
            pka_datas = res_json["gen_datas"]
            return pka_datas
        else:
            raise RuntimeError("Error for pKa prediction")
    else:
        raise RuntimeError("Error for pKa prediction")

smi = "CCn1cc(c(=O)c2c1cc(c(c2)F)N3CCNCC3)C(=O)O"
# smi = "COc(n1)ccc(c12)nccc2NC(=O)C3CCC(CC3)CCc(c4)ccc(c45)OCCO5"
# smi = "COc(n1)ccc(c12)nccc2NC(=O)C3(CC3)CCc(c4)ccc(c45)OCCO5"
data_pka = predict_pka(smi)
# print(type(data_pka))
data_pka

KeyError: 'ifjson'

In [31]:
def response_to_sdf(data_pka, i, smi, chembl_id):
    str = f'{i}{data_pka["MolBlock"]}>  <SMILES>  ({i})\n{smi}\n\n>  <chembl_id>  ({i})\n{chembl_id}'
    str += "test"
    _i = 1
    for type in ['Acid','Base']:
        for idx, pka in data_pka[type].items():
            str += f">  <pka_{_i}>  ({i})\n{pka}\n\n>  <pka_type_{_i}>  ({i})\n{type}\n\n>  <pka_atom_{_i}>  ({i})\n{idx}\n\n"
            _i += 1
    str += '$$$$\n'
    
    return str
print(response_to_sdf(data_pka,0, smi, 'CHEMBL503643'))

0
     RDKit          2D

 62 66  0  0  0  0  0  0  0  0999 V2000
    1.5000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.7500   -1.2990    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -0.7500   -1.2990    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.5000   -2.5981    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -3.0000   -2.5981    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -3.7500   -1.2990    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -5.2500   -1.2990    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
   -6.0000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -3.0000    0.0000    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -1.5000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.7500    1.2990    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.5000    2.5981    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -0.7500    3.8971    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.2268    3.7772    

In [49]:
input_file = "../../../chembl_data/chembl_29.sdf"
def load_sdf(input_file):
    suppl = Chem.SDMolSupplier(str(input_file), removeHs=False)
    # df_rows = []
    sdf_str = ""
    for ind, mol in enumerate(suppl):
        if mol is not None:
            smi = Chem.MolToSmiles(mol)
            print(smi)
            data_pka = predict_pka(smi)
            chembl_id =mol.GetPropsAsDict()['chembl_id']
            sdf_str += response_to_sdf(data_pka,ind, smi, chembl_id)
            print(f"{ind} finished")
        if ind > 2:
            break
    return sdf_str


In [50]:
sdf = load_sdf(input_file)

Cc1cc(-c2csc(N=C(N)N)n2)cn1C
0 finished
[H][C@@](NC(=O)[C@H](CC(C)C)NC(=O)[C@@]([H])(NC(=O)[C@@H](N)CCSC)[C@@H](C)O)(C(=O)NCC(=O)N[C@@H](C)C(=O)N[C@@H](C)C(=O)N[C@@H](Cc1c[nH]cn1)C(=O)N[C@@H](CC(N)=O)C(=O)NCC(=O)N[C@@H](C)C(=O)N[C@@H](C)C(=O)N[C@@H](CCC(N)=O)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CCCN=C(N)N)C(=O)N[C@@H](CCC(N)=O)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CCCN=C(N)N)C(=O)NCC(=O)N[C@@H](CCC(N)=O)C(=O)N[C@@H](CC(C)C)C(=O)NCC(=O)N1CCC[C@@]1([H])C(=O)N1CCC[C@@]1([H])C(=O)NCC(=O)N[C@@H](CO)C(=O)N[C@@H](CCCN=C(N)N)C(N)=O)[C@@H](C)CC


KeyError: 'ifjson'

In [13]:
from rdkit import Chem
import pandas as pd

def load_sdf_files(input_files):
  """Load SDF file into dataframe."""
  dataframes = []
  for input_file in input_files:
    # Tasks are stored in .sdf.csv file
    raw_df = next(load_csv_files([input_file+".csv"], shard_size=None))
    # Structures are stored in .sdf file
    print("Reading structures from %s." % input_file)
    suppl = Chem.SDMolSupplier(str(input_file), removeHs=False)
    df_rows = []
    for ind, mol in enumerate(suppl):
      if mol is not None:
        smiles = Chem.MolToSmiles(mol)
        df_rows.append([ind,smiles,mol])
    mol_df = pd.DataFrame(df_rows, columns=('mol_id', 'smiles', 'mol'))
    dataframes.append(pd.concat([mol_df, raw_df], axis=1, join='inner'))
  return dataframes

In [19]:
input_file = "../../../chembl_data/chembl_29.sdf"

In [22]:
def load_sdf(input_file):
    suppl = Chem.SDMolSupplier(str(input_file), removeHs=False)
    df_rows = []
    for ind, mol in enumerate(suppl):
        if mol is not None:
            smiles = Chem.MolToSmiles(mol)
            df_rows.append([ind,smiles,mol])
            print(mol.GetPropsAsDict()['chembl_id'])
        if ind > 100:
            break
    mol_df = pd.DataFrame(df_rows, columns=('mol_id', 'smiles', 'mol'))
    return mol_df

In [23]:
load_sdf(input_file)

CHEMBL153534
CHEMBL440060
CHEMBL440245
CHEMBL440249
CHEMBL405398
CHEMBL503634
CHEMBL503643
CHEMBL503865
CHEMBL503870
CHEMBL503873
CHEMBL504077
CHEMBL501567
CHEMBL501665
CHEMBL501667
CHEMBL501671
CHEMBL501672
CHEMBL403325
CHEMBL155287
CHEMBL265174
CHEMBL264472
CHEMBL405225
CHEMBL409812
CHEMBL499520
CHEMBL1082532
CHEMBL441562
CHEMBL441569
CHEMBL501674
CHEMBL501943
CHEMBL501944
CHEMBL501918
CHEMBL501922
CHEMBL501923
CHEMBL500195
CHEMBL500202
CHEMBL500702
CHEMBL500704
CHEMBL500709
CHEMBL501093
CHEMBL501094
CHEMBL501101
CHEMBL501238
CHEMBL501493
CHEMBL265653
CHEMBL505931
CHEMBL505936
CHEMBL505943
CHEMBL506057
CHEMBL264137
CHEMBL440444
CHEMBL1203224
CHEMBL438015
CHEMBL438018
CHEMBL441620
CHEMBL444522
CHEMBL444524
CHEMBL444732
CHEMBL444906
CHEMBL444987
CHEMBL440252
CHEMBL440255
CHEMBL1203109
CHEMBL265216
CHEMBL263058
CHEMBL263810
CHEMBL4116085
CHEMBL266960
CHEMBL443992
CHEMBL409297
CHEMBL405416
CHEMBL405667
CHEMBL411601
CHEMBL441131
CHEMBL155374
CHEMBL217730
CHEMBL261894
CHEMBL262664
CHEMBL43

,mol_id,smiles,mol
0,0,Cc1cc(-c2csc(N=C(N)N)n2)cn1C,<rdkit.Chem.rdchem.Mol object at 0x7fdf37cda6a0>
1,1,[H][C@@](NC(=O)[C@H](CC(C)C)NC(=O)[C@@]([H])(N...,<rdkit.Chem.rdchem.Mol object at 0x7fdf37ce1b20>
2,2,CCCC[C@@H]1NC(=O)[C@@H](NC(=O)[C@H](CC(C)C)NC(...,<rdkit.Chem.rdchem.Mol object at 0x7fdf37cb9c40>
3,3,[H][C@@]1(c2ccc(O[C@H]3O[C@H](CO)[C@@H](O)[C@H...,<rdkit.Chem.rdchem.Mol object at 0x7fdf37ce1d00>
4,4,Brc1cccc(Nc2ncnc3ccncc23)c1NCCN1CCOCC1,<rdkit.Chem.rdchem.Mol object at 0x7fdf37ce1220>
...,...,...,...
97,97,COC(=O)c1cc(CCc2ccc(OC)cc2OC)ccc1O,<rdkit.Chem.rdchem.Mol object at 0x7fdf37c524c0>
98,98,C=C(CO)[C@]12C=C3C(=O)C[C@@H](C(C)C)[C@]3(C)C[...,<rdkit.Chem.rdchem.Mol object at 0x7fdf37c52580>
99,99,CC[C@H](C)[C@H](NC(=O)[C@@H](N)CCCCN)C(=O)N[C@...,<rdkit.Chem.rdchem.Mol object at 0x7fdf37c525e0>
100,100,[H][C@]1(C(C)(O)C(O)CC(=C)C(C)C)CC[C@@]2([H])C...,<rdkit.Chem.rdchem.Mol object at 0x7fdf37c52640>


In [8]:
from chembl_webresource_client.new_client import new_client
import json
molecule = new_client.molecule
res = molecule

In [12]:
json_chembl = []
for i in tqdm(range(len(res))):
    
    json_chembl.append(res[i])

In [ ]:
with open('run_data/chembl_mols.json', 'w') as json_file:
    json.dump(json_chembl, json_file)

In [10]:
res[2000000]

{'atc_classifications': [],
 'availability_type': -1,
 'biotherapeutic': None,
 'black_box_warning': 0,
 'chebi_par_id': None,
 'chirality': -1,
 'cross_references': [],
 'dosed_ingredient': False,
 'first_approval': None,
 'first_in_class': -1,
 'helm_notation': None,
 'indication_class': None,
 'inorganic_flag': -1,
 'max_phase': 0,
 'molecule_chembl_id': 'CHEMBL4473180',
 'molecule_hierarchy': {'molecule_chembl_id': 'CHEMBL4473180',
  'parent_chembl_id': 'CHEMBL4473180'},
 'molecule_properties': {'alogp': '3.06',
  'aromatic_rings': 4,
  'cx_logd': '2.98',
  'cx_logp': '2.98',
  'cx_most_apka': '11.46',
  'cx_most_bpka': '3.66',
  'full_molformula': 'C19H14FN5OS',
  'full_mwt': '379.42',
  'hba': 6,
  'hba_lipinski': 6,
  'hbd': 3,
  'hbd_lipinski': 4,
  'heavy_atoms': 27,
  'molecular_species': 'NEUTRAL',
  'mw_freebase': '379.42',
  'mw_monoisotopic': '379.0903',
  'num_lipinski_ro5_violations': 0,
  'num_ro5_violations': 0,
  'psa': '100.71',
  'qed_weighted': '0.46',
  'ro3_pass

In [26]:
for i in range(1000):
    res[i]["molecule_structures"]["canonical_smiles"]

TypeError: 'NoneType' object is not subscriptable

In [14]:
data_pka.keys()
print(data_pka["MolBlock"])


     RDKit          2D

 62 66  0  0  0  0  0  0  0  0999 V2000
    1.5000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.7500   -1.2990    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -0.7500   -1.2990    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.5000   -2.5981    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -3.0000   -2.5981    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -3.7500   -1.2990    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -5.2500   -1.2990    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
   -6.0000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -3.0000    0.0000    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -1.5000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.7500    1.2990    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.5000    2.5981    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
   -0.7500    3.8971    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.2268    3.7772    0